<h1><span style="color:gray">ipyrad-analysis toolkit:</span> distance</h1>

Genetic distance matrices are used in many contexts to study the evolutionary divergence of samples or populations. The `ipa.distance` module provides a simple and convenient framework to implement several distance based metrics. 

Key features:

1. Filter SNPs to reduce missing data. 
2. Impute missing data using population allele frequencies.
3. Calculate pairwise genetic distances between samples (e.g., p-dist, JC, HKY, Fst)
4. (coming soon) sliding window measurements along chromosomes. 

### required software

In [29]:
# conda install ipyrad -c conda-forge -c bioconda
# conda install ipcoal -c conda-forge

In [30]:
import ipyrad.analysis as ipa
import ipcoal
import toyplot
import toytree

### Species tree model

In [31]:
# generate and draw an imbalanced 5 tip tree
tree = toytree.rtree.imbtree(ntips=5, treeheight=500000)
tree.draw(ts='p');

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t98a06c8a97114358872e4d84afee04c6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 r0 r1 r2 r3 r4 0 250000 500000

### Coalescent simulations 
The SNPs output is saved to an HDF5 database file.

In [9]:
# setup a model to simulate 8 haploid samples per species
model = ipcoal.Model(tree=tree, Ne=1e4, nsamples=8)
model.sim_loci(1000, 50)
model.write_snps_to_hdf5(name="test-dist", outdir="/tmp", diploid=True)

wrote 1044 SNPs to /tmp/test-dist.snps.hdf5


In [32]:
# the path to the HDF5 formatted snps file
SNPS = "/tmp/test-dist.snps.hdf5"

### [optional] Build an IMAP dictionary
A dictionary mapping of population names to sample names. 

In [33]:
from itertools import groupby

# load sample names from SNPs file
tool = ipa.snps_extracter(SNPS)

# group names by prefix before '-'
groups = groupby(tool.names, key=lambda x: x.split("-")[0])

# arrange into a dictionary
IMAP = {i[0]: list(i[1]) for i in groups}

# show the dict
IMAP

{'r0': ['r0-0', 'r0-1', 'r0-2', 'r0-3'],
 'r1': ['r1-0', 'r1-1', 'r1-2', 'r1-3'],
 'r2': ['r2-0', 'r2-1', 'r2-2', 'r2-3'],
 'r3': ['r3-0', 'r3-1', 'r3-2', 'r3-3'],
 'r4': ['r4-0', 'r4-1', 'r4-2', 'r4-3']}

### calculate distances with missing values filtered and/or imputed, and *corrected*
The correction applies a model of sequence substitution where more complex models can apply a greater penalty for unobserved changes (e.g., HKY or GTR). This allows you to use either SNPs or SEQUENCES as input. Here we are using SNPs. More on this later... (TODO). 

In [25]:
dist = ipa.distance(
    data=SNPS, 
    imap=IMAP, 
    minmap={i: 1 for i in IMAP},
    mincov=0.5,
    impute_method=None,
)

In [34]:
# infer the distance matrix from sequence data
dist.run()

Samples: 20
Sites before filtering: 1044
Filtered (indels): 0
Filtered (bi-allel): 9
Filtered (mincov): 0
Filtered (minmap): 0
Filtered (subsample invariant): 0
Filtered (combined): 9
Sites after filtering: 1035
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'None'; (0, 1, 2) = 100.0%, 0.0%, 0.0%


In [35]:
# show the first few data cells
dist.dists.iloc[:5, :12]

,r0-0,r0-1,r0-2,r0-3,r1-0,r1-1,r1-2,r1-3,r2-0,r2-1,r2-2,r2-3
r0-0,0.000,0.033,0.045,0.036,0.510,0.499,0.510,0.518,0.911,0.915,0.892,0.908
r0-1,0.033,0.000,0.032,0.024,0.508,0.497,0.508,0.516,0.909,0.913,0.890,0.906
r0-2,0.045,0.032,0.000,0.031,0.509,0.498,0.509,0.517,0.910,0.914,0.891,0.907
r0-3,0.036,0.024,0.031,0.000,0.505,0.494,0.505,0.513,0.906,0.910,0.887,0.903
r1-0,0.510,0.508,0.509,0.505,0.000,0.043,0.050,0.046,0.990,0.994,0.969,0.984


### Infer a tree from distance matrix

In [37]:
tool = ipa.neighbor_joining(matrix=dist.dists)

ToytreeError: node idx or name 6 not in tree

### Draw tree and distance matrix

In [113]:
# create a canvas
canvas = toyplot.Canvas(width=500, height=450);

# add tree
axes = canvas.cartesian(bounds=("10%", "35%", "10%", "90%"))
gtree.draw(axes=axes, tip_labels=True, tip_labels_align=True)

# add matrix
table = canvas.table(
    rows=matrix.shape[0],
    columns=matrix.shape[1],
    margin=0,
    bounds=("40%", "95%", "9%", "91%"),
)

colormap = toyplot.color.brewer.map("BlueRed")

# apply a color to each cell in the table
for ridx in range(matrix.shape[0]):
    for cidx in range(matrix.shape[1]):
        cell = table.cells.cell[ridx, cidx]
        cell.style = {
            "fill": colormap.colors(matrix.iloc[ridx, cidx], 0, 1),
        }

<svg class="toyplot-canvas-Canvas" height="450.0px" id="t19afc0e9e9b04240ba7729b7204f739b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 450.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r1-3 r1-0 r1-1 r1-2 r0-2 r0-0 r0-3 r0-1 r2-3 r2-1 r2-2 r2-0 r3-3 r3-0 r3-2 r3-1 r4-3 r4-1 r4-0 r4-2 r5-1 r5-0 r5-3 r5-2 r6-2 r6-0 r6-3 r6-1 r7-2 r7-0 r7-1 r7-3 -150000 -100000 -50000 0 50000 0 10 20 30

In [99]:
dist.dists

,r0-0,r0-1,r1-0,r1-1,r2-0,r2-1,r3-0,r3-1,r4-0,r4-1,r5-0,r5-1,r6-0,r6-1,r7-0,r7-1
r0-0,0.000,0.079,0.556,0.587,0.873,0.778,0.714,0.794,0.730,0.730,0.825,0.778,0.778,0.730,0.746,0.841
r0-1,0.079,0.000,0.508,0.540,0.825,0.730,0.667,0.746,0.683,0.683,0.778,0.730,0.730,0.683,0.698,0.794
r1-0,0.556,0.508,0.000,0.063,0.825,0.730,0.667,0.746,0.683,0.683,0.778,0.730,0.730,0.683,0.698,0.794
r1-1,0.587,0.540,0.063,0.000,0.857,0.762,0.698,0.778,0.714,0.714,0.810,0.762,0.762,0.714,0.730,0.825
r2-0,0.873,0.825,0.825,0.857,0.000,0.063,0.476,0.556,0.746,0.746,0.841,0.794,0.794,0.746,0.762,0.857
r2-1,0.778,0.730,0.730,0.762,0.063,0.000,0.381,0.460,0.651,0.651,0.746,0.698,0.698,0.651,0.667,0.762
r3-0,0.714,0.667,0.667,0.698,0.476,0.381,0.000,0.079,0.587,0.587,0.683,0.635,0.635,0.587,0.603,0.698
r3-1,0.794,0.746,0.746,0.778,0.556,0.460,0.079,0.000,0.667,0.667,0.762,0.714,0.714,0.667,0.683,0.778
r4-0,0.730,0.683,0.683,0.714,0.746,0.651,0.587,0.667,0.000,0.095,0.317,0.270,0.397,0.286,0.492,0.587
r4-1,0.730,0.683,0.683,0.714,0.746,0.651,0.587,0.667,0.095,0.000,0.254,0.206,0.397,0.286,0.492,0.587


In [39]:


# style the gaps between cells
table.body.gaps.columns[:] = 3
table.body.gaps.rows[:] = 3

# hide axes coordinates
axes.show = False

KeyError: (0, 0)

<svg class="toyplot-canvas-Canvas" height="350.0px" id="t320b72e093a14118b1045ef754b39d03" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 350.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -100000 -50000 0 0 10 20 30

In [5]:
# load the snp data into distance tool with arguments
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
)
dist.run()

Samples: 29
Sites before filtering: 349914
Filtered (indels): 0
Filtered (bi-allel): 13379
Filtered (mincov): 30459
Filtered (minmap): 111825
Filtered (combined): 120177
Sites after filtering: 229737
Sites containing missing values: 219551 (95.57%)
Missing values in SNP matrix: 814369 (12.22%)
Imputation: 'sampled'; (0, 1, 2) = 77.3%, 10.7%, 12.0%


#### save results

In [6]:
# save to a CSV file
dist.dists.to_csv("distances.csv")

In [7]:
# show the upper corner 
dist.dists.head()

,BJSB3,BJSL25,BJVL19,BZBB1,CRL0001,CRL0030,CUCA4,CUMM5,CUSV6,CUVN10,...,FLWO6,HNDA09,LALC2,MXED8,MXGT4,MXSA3017,SCCU3,TXGR3,TXMD3,TXWV2
BJSB3,0.000000,0.250447,0.253472,0.592255,0.530145,0.572576,0.601853,0.597044,0.591990,0.579937,...,0.594005,0.582000,0.568137,0.464618,0.443942,0.579789,0.603638,0.487945,0.487936,0.590440
BJSL25,0.250447,0.000000,0.235900,0.558630,0.494291,0.537193,0.566156,0.559675,0.554665,0.542768,...,0.558769,0.548897,0.532239,0.435050,0.412694,0.547182,0.567323,0.453606,0.457105,0.554882
BJVL19,0.253472,0.235900,0.000000,0.567897,0.502775,0.547391,0.576355,0.569360,0.563000,0.554621,...,0.564728,0.555927,0.539913,0.441844,0.417060,0.556449,0.575336,0.464118,0.465476,0.562278
BZBB1,0.592255,0.558630,0.567897,0.000000,0.280691,0.280569,0.422670,0.422962,0.426266,0.394242,...,0.559152,0.285883,0.532918,0.525701,0.542381,0.317450,0.576455,0.552554,0.551579,0.563571
CRL0001,0.530145,0.494291,0.502775,0.280691,0.000000,0.239596,0.347859,0.322277,0.342836,0.213266,...,0.470064,0.262217,0.451717,0.466429,0.477764,0.299538,0.492224,0.487327,0.484123,0.482726


### Draw the matrix

In [8]:
toyplot.matrix(
    dist.dists, 
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(dist.names)),
        sorted(dist.names),
));

<svg class="toyplot-canvas-Canvas" height="600px" id="td59966f185b849fa9dae639ce90d3bbf" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.250447 0.253472 0.592255 0.530145 0.572576 0.601853 0.597044 0.591990 0.579937 0.571636 0.548179 0.589169 0.590719 0.569939 0.596526 0.585417 0.583654 0.555365 0.594005 0.582000 0.568137 0.464618 0.443942 0.579789 0.603638 0.487945 0.487936 0.590440 BJSB3 1 0.250447 0.000000 0.235900 0.558630 0.494291 0.537193 0.566156 0.559675 0.554665 0.542768 0.535295 0.510732 0.552984 0.556475 0.536278 0.564023 0.548762 0.548488 0.519181 0.558769 0.548897 0.532239 0.435050 0.412694 0.547182 0.567323 0.453606 0.457105 0.554882 BJSL25 2 0.253472 0.235900 0.000000 0.567897 0.502775 0.547391 0.576355 0.569360 0.563000 0.554621 0.540784 0.516717 0.561068 0.561285 0.539138 0.569016 0.557202 0.556223 0.526054 0.564728 0.555927 0.539913 0.441844 0.417060 0.556449 0.575336 0.464118 0.465476 0.562278 BJVL19 3 0.592255 0.558630 0.567897 0.000000 0.280691 0.280569 0.422670 0.422962 0.426266 0.394242 0.539500 0.518872 0.561612 0.566617 0.548910 0.576220 0.552993 0.555148 0.520330 0.559152 0.285883 0.532918 0.525701 0.542381 0.317450 0.576455 0.552554 0.551579 0.563571 BZBB1 4 0.530145 0.494291 0.502775 0.280691 0.000000 0.239596 0.347859 0.322277 0.342836 0.213266 0.449018 0.435598 0.471417 0.478495 0.465663 0.492389 0.473498 0.468819 0.429630 0.470064 0.262217 0.451717 0.466429 0.477764 0.299538 0.492224 0.487327 0.484123 0.482726 CRL0001 5 0.572576 0.537193 0.547391 0.280569 0.239596 0.000000 0.398194 0.401516 0.401651 0.368356 0.528639 0.508177 0.549428 0.550264 0.532748 0.561555 0.542647 0.538237 0.507032 0.549842 0.256123 0.522258 0.508216 0.523956 0.306529 0.562069 0.536096 0.536017 0.549550 CRL0030 6 0.601853 0.566156 0.576355 0.422670 0.347859 0.398194 0.000000 0.290293 0.368230 0.359267 0.481960 0.477794 0.502531 0.519934 0.504177 0.541611 0.511402 0.507263 0.461972 0.500638 0.405442 0.487470 0.538111 0.544353 0.433422 0.532679 0.547622 0.544975 0.523372 CUCA4 7 0.597044 0.559675 0.569360 0.422962 0.322277 0.401516 0.290293 0.000000 0.288430 0.288391 0.472249 0.469136 0.494753 0.511215 0.499898 0.532909 0.502927 0.495323 0.452705 0.490195 0.406090 0.481416 0.530272 0.539299 0.431955 0.523211 0.542986 0.539016 0.513217 CUMM5 8 0.591990 0.554665 0.563000 0.426266 0.342836 0.401651 0.368230 0.288430 0.000000 0.358423 0.469511 0.460104 0.489560 0.501661 0.491510 0.524522 0.496429 0.493308 0.448774 0.487518 0.409412 0.474403 0.530111 0.533976 0.431950 0.514736 0.536087 0.532587 0.508268 CUSV6 9 0.579937 0.542768 0.554621 0.394242 0.213266 0.368356 0.359267 0.288391 0.358423 0.000000 0.462707 0.454938 0.488354 0.501308 0.490757 0.517265 0.493342 0.488889 0.446959 0.487079 0.374576 0.467826 0.515429 0.523986 0.401285 0.512904 0.525401 0.522493 0.503824 CUVN10 10 0.571636 0.535295 0.540784 0.539500 0.449018 0.528639 0.481960 0.472249 0.469511 0.462707 0.000000 0.381902 0.344150 0.406961 0.403531 0.441487 0.418609 0.386316 0.313323 0.338783 0.535086 0.398334 0.512769 0.511646 0.528957 0.444404 0.499993 0.493264 0.433504 FLAB109 11 0.548179 0.510732 0.516717 0.518872 0.435598 0.508177 0.477794 0.469136 0.460104 0.454938 0.381902 0.000000 0.404798 0.418979 0.404650 0.443329 0.321137 0.403165 0.356908 0.399779 0.514432 0.305989 0.481337 0.480937 0.509766 0.329999 0.460405 0.452517 0.301062 FLBA140 12 0.589169 0.552984 0.561068 0.561612 0.471417 0.549428 0.502531 0.494753 0.489560 0.488354 0.344150 0.404798 0.000000 0.425530 0.422979 0.459856 0.439102 0.413329 0.327662 0.357483 0.553298 0.416676 0.530415 0.529179 0

### Draw matrix reordered to match groups in imap

In [9]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [10]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="te56369b7ffaf44ae952e6df25bc58784" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.299386 0.322760 0.318730 0.301062 0.455099 0.456853 0.493852 0.468579 0.454450 0.407579 0.451695 0.433504 0.554882 0.590440 0.562278 0.523103 0.522571 0.501029 0.496041 0.503824 0.523372 0.508268 0.513217 0.549550 0.482726 0.558321 0.563571 0.552218 TXWV2 1 0.299386 0.000000 0.338578 0.324406 0.305989 0.417046 0.419924 0.461119 0.435054 0.416676 0.371599 0.416272 0.398334 0.532239 0.568137 0.539913 0.497900 0.494670 0.474416 0.465728 0.467826 0.487470 0.474403 0.481416 0.522258 0.451717 0.527538 0.532918 0.523690 LALC2 2 0.322760 0.338578 0.000000 0.349704 0.329999 0.461645 0.464444 0.499615 0.475457 0.463957 0.416607 0.459121 0.444404 0.567323 0.603638 0.575336 0.538729 0.535787 0.515455 0.505678 0.512904 0.532679 0.514736 0.523211 0.562069 0.492224 0.568707 0.576455 0.564990 SCCU3 3 0.318730 0.324406 0.349704 0.000000 0.321137 0.434754 0.441801 0.479923 0.453170 0.439102 0.389358 0.436869 0.418609 0.548762 0.585417 0.557202 0.515407 0.515059 0.494962 0.489573 0.493342 0.511402 0.496429 0.502927 0.542647 0.473498 0.548109 0.552993 0.543983 FLSF33 4 0.301062 0.305989 0.329999 0.321137 0.000000 0.403165 0.404650 0.443329 0.418979 0.404798 0.356908 0.399779 0.381902 0.510732 0.548179 0.516717 0.481337 0.480937 0.460405 0.452517 0.454938 0.477794 0.460104 0.469136 0.508177 0.435598 0.514432 0.518872 0.509766 FLBA140 5 0.455099 0.417046 0.461645 0.434754 0.403165 0.000000 0.372221 0.399923 0.383216 0.413329 0.372552 0.414465 0.386316 0.548488 0.583654 0.556223 0.524387 0.523377 0.510958 0.509147 0.488889 0.507263 0.493308 0.495323 0.538237 0.468819 0.546468 0.555148 0.540165 FLSF47 6 0.456853 0.419924 0.464444 0.441801 0.404650 0.372221 0.000000 0.377876 0.375473 0.422979 0.386538 0.425430 0.403531 0.536278 0.569939 0.539138 0.516390 0.514928 0.506880 0.500995 0.490757 0.504177 0.491510 0.499898 0.532748 0.465663 0.543313 0.548910 0.535512 FLMO62 7 0.493852 0.461119 0.499615 0.479923 0.443329 0.399923 0.377876 0.000000 0.389236 0.459856 0.427106 0.466111 0.441487 0.564023 0.596526 0.569016 0.545502 0.545328 0.541023 0.535774 0.517265 0.541611 0.524522 0.532909 0.561555 0.492389 0.567262 0.576220 0.562291 FLSA185 8 0.468579 0.435054 0.475457 0.453170 0.418979 0.383216 0.375473 0.389236 0.000000 0.425530 0.390481 0.429896 0.406961 0.556475 0.590719 0.561285 0.537371 0.532174 0.527625 0.518937 0.501308 0.519934 0.501661 0.511215 0.550264 0.478495 0.558730 0.566617 0.551296 FLCK216 9 0.454450 0.416676 0.463957 0.439102 0.404798 0.413329 0.422979 0.459856 0.425530 0.000000 0.327662 0.357483 0.344150 0.552984 0.589169 0.561068 0.530415 0.529179 0.518615 0.513069 0.488354 0.502531 0.489560 0.494753 0.549428 0.471417 0.553298 0.561612 0.551827 FLCK18 10 0.407579 0.371599 0.416607 0.389358 0.356908 0.372552 0.386538 0.427106 0.390481 0.327662 0.000000 0.324889 0.313323 0.519181 0.555365 0.526054 0.494862 0.496794 0.480127 0.473798 0.446959 0.461972 0.448774 0.452705 0.507032 0.429630 0.515159 0.520330 0.507028 FLSF54 11 0.451695 0.416272 0.459121 0.436869 0.399779 0.414465 0.425430 0.466111 0.429896 0.357483 0.324889 0.000000 0.338783 0.558769 0.594005 0.564728 0.533397 0.531612 0.520970 0.516826 0.487079 0.500638 0.487518 0.490195 0.549842 0.470064 0.555305 0.559152 0.552667 FLWO6 12 0.433504 0.398334 0.444404 0.418609 0.381902 0.386316 0.403531 0.441487 0.406961 0.344150 0.313323 0.338783 0.000000 0.535295 0.571636 0.540784 0.512769 0.511646 0.499993 0.493264 0.462707 0.481960 0.469511 0.472249 0